In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% }</style>"))

In [14]:
import warnings
warnings.filterwarnings("ignore")

## Step 1: pip install required packages if not installed already

In [3]:
# !pip install h2o
import benchmark_h2o
from benchmark_h2o import load_data, run_h2o

## Step 2: download data using fdb
Example: https://github.com/amazon-research/fraud-dataset-benchmark/blob/main/scripts/examples/Test_FDB_Loader.ipynb

In [4]:
# This is where datasets are stored: {BASE_PATH}/{dataset}/
BASE_PATH = "/home/ec2-user/SageMaker/official-dataset-names"
dataset = "IEEE-CIS Fraud Detection"

Make sure three files are downloaded:
1. {BASE_PATH}/{dataset}/train.csv
2. {BASE_PATH}/{dataset}/test.csv
3. {BASE_PATH}/{dataset}/test_labels.csv

## Step 3: look at data

In [5]:
features, df_train, df_test = load_data(dataset, BASE_PATH)

INFO: benchmark_utils.py: IEEE-CIS Fraud Detection
INFO: benchmark_utils.py: (313060, 194)
INFO: benchmark_utils.py: (27330, 71)
INFO: benchmark_utils.py: (29527, 2)
INFO: benchmark_utils.py: (27329, 72)
INFO: benchmark_utils.py: 67
INFO: benchmark_utils.py: ['transactionamt', 'productcd', 'card1', 'card2', 'card3', 'card5', 'card6', 'addr1', 'dist1', 'p_emaildomain', 'r_emaildomain', 'c1', 'c2', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'v62', 'v70', 'v76', 'v78', 'v82', 'v91', 'v127', 'v130', 'v139', 'v160', 'v165', 'v187', 'v203', 'v207', 'v209', 'v210', 'v221', 'v234', 'v257', 'v258', 'v261', 'v264', 'v266', 'v267', 'v271', 'v274', 'v277', 'v283', 'v285', 'v289', 'v291', 'v294', 'id_01', 'id_02', 'id_05', 'id_06', 'id_09', 'id_13', 'id_17', 'id_19', 'id_20', 'devicetype', 'deviceinfo']


In [6]:
df_train.head()

,EVENT_LABEL,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,addr2,dist1,dist2,p_emaildomain,r_emaildomain,c1,c2,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,d1,d2,d3,d4,d5,d10,d11,d15,m1,m2,m3,m4,m6,m7,m8,m9,v1,v3,v4,v6,v8,v11,v13,v14,v17,v20,v23,v26,v27,v30,v36,v37,v40,v41,v44,v47,v48,v54,v56,v59,v62,v65,v67,v68,v70,v76,v78,v80,v82,v86,v88,v89,v91,v107,v108,v111,v115,v117,v120,v121,v123,v124,v127,v129,v130,v136,v138,v139,v142,v147,v156,v160,v162,v165,v166,v169,v171,v173,v175,v176,v178,v180,v182,v185,v187,v188,v198,v203,v205,v207,v209,v210,v215,v218,v220,v221,v223,v224,v226,v228,v229,v234,v235,v238,v240,v250,v252,v253,v257,v258,v260,v261,v264,v266,v267,v271,v274,v277,v281,v283,v284,v285,v286,v289,v291,v294,v296,v297,v301,v303,v305,v307,v309,v310,v314,v320,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_15,id_16,id_17,id_18,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,devicetype,deviceinfo,ENTITY_ID,EVENT_TIMESTAMP,ENTITY_TYPE,EVENT_ID,LABEL_TIMESTAMP
0,0,68.500,W,13926.000,NaN,150.000,142.000,credit,315.000,87.000,19.000,NaN,NaN,NaN,1.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,2.000,0.000,1.000,1.000,14.000,NaN,13.000,NaN,NaN,12.000,12.000,-1.000,T,T,T,M2,T,NaN,NaN,NaN,1.000,1.000,1.000,1.000,1.000,0.000,1.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,0.000,0.000,1.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,117.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1.000,0.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,1.000,117.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13926.0_315.0_-13.0,2021-01-02 00:00:00,user,2987000.000,2022-01-01T20:30:04Z
1,0,29.000,W,2755.000,404.000,150.000,102.000,credit,325.000,87.000,NaN,NaN,gmail.com,NaN,1.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,1.000,0.000,NaN,NaN,-1.000,NaN,-1.000,NaN,-1.000,NaN,NaN,NaN,M0,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,0.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,0.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2755.0_325.0_1.0,2021-01-02 00:00:01,user,2987001.000,2022-01-01T20:30:04Z
2,0,59.000,W,4663.000,490.000,150.000,166.000,debit,330.000,87.000,287.000,NaN,outlook.com,NaN,1.000,1.000,0.000,0.000,1.000,0.000,0.000,1.000,0.000,1.000,0.000,1.000,1.000,0.000,NaN,NaN,-1.001,NaN,-1.001,313.999,313.999,T,T,T,M0,F,F,F,F,1.000,1.000,1.000,1.000,1.000,0.000,1.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,0.000,1.000,1.000,1.000,0.000,1.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,0.000,1.000,1.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [7]:
df_test.head()

,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,p_emaildomain,r_emaildomain,c1,c2,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,v62,v70,v76,v78,v82,v91,v127,v130,v139,v160,v165,v187,v203,v207,v209,v210,v221,v234,v257,v258,v261,v264,v266,v267,v271,v274,v277,v283,v285,v289,v291,v294,id_01,id_02,id_05,id_06,id_09,id_13,id_17,id_19,id_20,devicetype,deviceinfo,EVENT_TIMESTAMP,ENTITY_ID,ENTITY_TYPE,EVENT_ID,EVENT_LABEL
0,125.000,S,15775.000,481.000,150.000,102.000,credit,330.000,NaN,NaN,yahoo.com,5.000,3.000,3.000,0.000,0.000,0.000,8.000,0.000,3.000,5.000,0.000,61.000,5.000,0.000,0.000,NaN,NaN,NaN,NaN,109411.000,2301.000,0.000,2401.000,66104.000,1.000,103183.000,877.000,1961.000,465.000,0.000,73.000,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,1.000,26.000,1.000,2.000,926.000,-10.000,1411.000,6.000,0.000,0.000,52.000,166.000,633.000,533.000,desktop,Windows,2021-06-21 23:11:15,15775.0_330.0_129.0,user,3548013.000,0
1,125.000,S,15775.000,481.000,150.000,102.000,credit,330.000,NaN,NaN,yahoo.com,5.000,3.000,3.000,0.000,0.000,0.000,8.000,0.000,3.000,5.000,0.000,61.000,5.000,0.000,0.000,NaN,NaN,NaN,NaN,109536.000,2301.000,0.000,2401.000,66229.000,1.000,103308.000,877.000,1961.000,465.000,0.000,73.000,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,1.000,26.000,1.000,2.000,927.000,-10.000,693.000,6.000,0.000,0.000,52.000,166.000,633.000,533.000,desktop,Windows,2021-06-21 23:11:29,15775.0_330.0_129.0,user,3548014.000,0
2,125.000,S,15775.000,481.000,150.000,102.000,credit,330.000,NaN,NaN,yahoo.com,5.000,3.000,3.000,0.000,0.000,0.000,8.000,0.000,3.000,5.000,0.000,61.000,5.000,0.000,0.000,NaN,NaN,NaN,NaN,109661.000,2301.000,0.000,2401.000,66354.000,1.000,103433.000,877.000,1961.000,465.000,0.000,73.000,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,1.000,26.000,1.000,2.000,928.000,-10.000,1116.000,6.000,0.000,0.000,52.000,166.000,633.000,533.000,desktop,Windows,2021-06-21 23:11:45,15775.0_330.0_129.0,user,3548015.000,0
3,125.000,S,15775.000,481.000,150.000,102.000,credit,330.000,NaN,NaN,yahoo.com,5.000,3.000,3.000,0.000,0.000,0.000,8.000,0.000,3.000,5.000,0.000,61.000,5.000,0.000,0.000,NaN,NaN,NaN,NaN,109786.000,2301.000,0.000,2401.000,66479.000,1.000,103558.000,877.000,1961.000,465.000,0.000,73.000,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,1.000,26.000,1.000,2.000,929.000,-10.000,1589.000,6.000,0.000,0.000,52.000,166.000,633.000,533.000,desktop,Windows,2021-06-21 23:12:00,15775.0_330.0_129.0,user,3548016.000,0
4,31.950,W,9500.000,321.000,150.000,226.000,debit,204.000,74.000,NaN,NaN,3.000,3.000,0.000,1.000,1.000,0.000,0.000,1.000,0.000,1.000,0.000,6.000,3.000,1.000,1.000,1.000,2.000,1.000,1.000,27.950,27.950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,1.000,1.000,1.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-21 23:12:11,9500.0_204.0_150.0,user,3548017.000,0


## Step 3: run H2O

1. The function run_h2o below also saves a leaderboard file (leaderboard_xxx.csv) and a test metrics file (test_metrics_xxx.joblib) into {BASE_PATH}/{dataset}/, respectively
2. H2O models are saved at {BASE_PATH}/{dataset}/H2OModels

In [8]:
# H2OStartupError: Your java is not supported: java version "1.7.0_261"; OpenJDK Runtime Environment (amzn-2.6.22.1.84.amzn1-x86_64 u261-b02); OpenJDK 64-Bit Server VM (build 24.261-b02, mixed mode)
# If you see this error above, you may need to run the following instructions:

# !sudo yum install -y java-1.8.0-openjdk.x86_64
# !sudo /usr/sbin/alternatives --set java /usr/lib/jvm/jre-1.8.0-openjdk.x86_64/bin/java
# !sudo /usr/sbin/alternatives --set javac /usr/lib/jvm/jre-1.8.0-openjdk.x86_64/bin/javac

https://swiftotter.com/technical/amazon-aws-jenkins-2-60-1-java-8-update#/

In [9]:
!java -version

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [13]:
import h2o
h2o.__version__

'3.36.1.2'

In [21]:
run_h2o(dataset, BASE_PATH, time_limit=3600)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312"; OpenJDK Runtime Environment (build 1.8.0_312-b07); OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)
  Starting server from /home/ec2-user/anaconda3/envs/mxnet_latest_p37/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpag6zcv5a
  JVM stdout: /tmp/tmpag6zcv5a/h2o_ec2_user_started_from_python.out
  JVM stderr: /tmp/tmpag6zcv5a/h2o_ec2_user_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_ec2_user_t9z3ig
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.64 Gb
H2O_cluster_total_cores:,64
H2O_cluster_allowed_cores:,64
H2O_cluster_status:,"locked, healthy"


INFO: benchmark_utils.py: IEEE-CIS Fraud Detection
INFO: benchmark_utils.py: (313060, 194)
INFO: benchmark_utils.py: (27330, 71)
INFO: benchmark_utils.py: (29527, 2)
INFO: benchmark_utils.py: (27329, 72)
INFO: benchmark_utils.py: 67
INFO: benchmark_utils.py: ['transactionamt', 'productcd', 'card1', 'card2', 'card3', 'card5', 'card6', 'addr1', 'dist1', 'p_emaildomain', 'r_emaildomain', 'c1', 'c2', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'v62', 'v70', 'v76', 'v78', 'v82', 'v91', 'v127', 'v130', 'v139', 'v160', 'v165', 'v187', 'v203', 'v207', 'v209', 'v210', 'v221', 'v234', 'v257', 'v258', 'v261', 'v264', 'v266', 'v267', 'v271', 'v274', 'v277', 'v283', 'v285', 'v289', 'v291', 'v294', 'id_01', 'id_02', 'id_05', 'id_06', 'id_09', 'id_13', 'id_17', 'id_19', 'id_20', 'devicetype', 'deviceinfo']


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
17:56:29.266: Project: AutoML_1_20220615_175629
17:56:29.267: 5-fold cross-validation will be used.
17:56:29.270: Setting stopping tolerance adaptively based on the training frame: 0.0017872537194430643
17:56:29.271: Build control seed: 10
17:56:29.274: training frame: Frame key: AutoML_1_20220615_175629_training_py_11_sid_9519    cols: 194    rows: 313060  chunks: 68    size: 137888486  checksum: -8673498857111412012
17:56:29.274: validation frame: NULL
17:56:29.274: leaderboard frame: NULL
17:56:29.275: blending frame: NULL
17:56:29.275: response column: EVENT_LABEL
17:56:29.275: fold column: null
17:56:29.275: weights column: null
17:56:29.289: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (6g, 30w)]}, {GLM 

INFO: benchmark_h2o.py: auc on test data: 0.8816474193300993
INFO: benchmark_h2o.py: tpr@1%fpr on test data: 0.43807763401109057


H2O session _sid_9519 closed.
